# 插补用户用电量数据缺失值

In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import lagrange

# 读取 missing_data.csv 表中的数据

missing_data = pd.read_excel("../data/missing_data.xls", header = None)

# 查询缺失值所在位置

print("缺失值所在位置的索引为：")
for i in missing_data.columns:
    for j in range(len(missing_data)):
        if (missing_data[i].isnull())[j]:
            print("(%d,%d)"%(j,i))

# 使用 Scipy 库中的 interpolate 模块中的 lagrange 对数据进行 lagrange 插值

def ployinterp_column(s, n): # s 为列向量，n 为被插值的位置
    y = s
    y = y[y.notnull()] # 剔除空值
    return lagrange(y.index, list(y))(n)

print("插值之前的数据为：\n", missing_data)
for i in missing_data.columns:
    for j in range(len(missing_data)):
        if (missing_data[i].isnull())[j]:
            missing_data[i][j] = ployinterp_column(missing_data[i], j)
print("插值之后的数据为：\n", missing_data)

# 查看数据中是否存在缺失值，若不存在则说明插值成功
print("数据表每列数据缺失值的个数为：")
print(missing_data.isnull().sum())

缺失值所在位置的索引为：
(5,0)
(12,0)
(3,1)
(10,1)
(19,1)
(20,1)
(4,2)
(7,2)
(10,2)
(16,2)
插值之前的数据为：
            0         1         2
0   235.8333  324.0343  478.3231
1   236.2708  325.6379  515.4564
2   238.0521  328.0897  517.0909
3   235.9063       NaN  514.8900
4   236.7604  268.8324       NaN
5        NaN  404.0480  486.0912
6   237.4167  391.2652  516.2330
7   238.6563  380.8241       NaN
8   237.6042  388.0230  435.3508
9   238.0313  206.4349  487.6750
10  235.0729       NaN       NaN
11  235.5313  400.0787  660.2347
12       NaN  411.2069  621.2346
13  234.4688  395.2343  611.3408
14  235.5000  344.8221  643.0863
15  235.6354  385.6432  642.3482
16  234.5521  401.6234       NaN
17  236.0000  409.6489  602.9347
18  235.2396  416.8795  589.3457
19  235.4896       NaN  556.3452
20  236.9688       NaN  538.3470
插值之后的数据为：
              0             1           2
0   235.833300  3.240343e+02  478.323100
1   236.270800  3.256379e+02  515.456400
2   238.052100  3.280897e+02  517.090900
3   235.9

# 合并线损、用电量趋势与线路告警数据

In [2]:
import pandas as pd

# 读取 ele_loss.csv 和 alarm.csv 表

ele_loss_data = pd.read_csv("../data/ele_loss.csv", sep = ",", encoding = "gbk")
alarm_data = pd.read_csv("../data/alarm.csv", sep = ",", encoding = "gbk")

# 查看两表的形状

print("ele_loss 表的形状为：", ele_loss_data.shape)
print("alarm_data 表的形状为：", alarm_data.shape)

# 以 ID 和 date 两个键值作为主键进行内连接

ele_loss_alarm_data = pd.merge(ele_loss_data, alarm_data, left_on = [ "ID","date" ], right_on = [ "ID","date" ])

# 查看合并后的数据

print(ele_loss_alarm_data)

ele_loss 表的形状为： (49, 4)
alarm_data 表的形状为： (25, 3)
         ID       date    ele      loss    alarm
0  21261001   2010/9/3  858.0  0.151048    电流不平衡
1  21261001   2010/9/6  883.5  0.162778  C相电流过负荷
2  21261001  2010/9/16  915.5  0.162028  C相电流过负荷
3  21261001  2010/9/16  915.5  0.162028  C相电流过负荷
4  21261001  2010/9/17  961.0  0.166650  A相电流过负荷
5  21261001  2010/9/17  961.0  0.166650    电流不平衡


# 标准化建模专家样本数据

In [3]:
import pandas as pd

# 读取 model.csv 数据

model_data = pd.read_excel("../data/model.xls", sep = ",", encoding = "gbk")

# 定义标准差标准化函数

def StandardScaler(data):
    data = (data - data.mean()) / data.std()
    return data

# 使用函数分别对 3 列数据进行标准化

cel_data_0 = StandardScaler(model_data["电量趋势下降指标"])
cel_data_1 = StandardScaler(model_data["线损指标"])
cel_data_2 = StandardScaler(model_data["告警类指标"])

data = pd.concat([ cel_data_0,cel_data_1,cel_data_2 ], axis = 1)

# 查看标准化后的数据

print("标准差标准化之后的电量趋势下降指标、线损指标、告警类指标的数据为：\n", data)

标准差标准化之后的电量趋势下降指标、线损指标、告警类指标的数据为：
      电量趋势下降指标      线损指标     告警类指标
0    0.612777  0.974549 -0.161216
1    0.612777 -1.022590  2.653616
2   -0.343352  0.974549 -0.161216
3    3.003099 -1.022590 -1.099494
4    0.134712  0.974549 -1.099494
5   -0.343352 -1.022590 -1.099494
6    1.090841 -1.022590  0.777061
7    0.134712  0.974549  1.715339
8    0.134712 -1.022590 -1.099494
9    0.612777  0.974549 -1.099494
10   3.481163  0.974549  0.777061
11   3.481163  0.974549  1.715339
12  -0.343352 -1.022590  1.715339
13   0.612777 -1.022590  0.777061
14   0.134712 -1.022590 -1.099494
15  -1.299481 -1.022590  1.715339
16   3.003099 -1.022590  1.715339
17  -1.299481 -1.022590  0.777061
18   2.525034  0.974549  2.653616
19  -0.343352 -1.022590  2.653616
20   0.134712 -1.022590 -0.161216
21   2.046970 -1.022590 -1.099494
22   1.568906 -1.022590 -1.099494
23   0.612777  0.974549  0.777061
24   2.046970 -1.022590 -1.099494
25  -0.343352  0.974549 -1.099494
26   1.090841  0.974549 -1.099494
27  -0.821417